In [62]:
from references_tbl.manual.func import *
from references_tbl.manual.create_tbl import *
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="weather_processing_script")

In [2]:
location_df = handle_location_data('location_data.xlsx')

In [3]:
location_df

,postal_code,province_(thai),province,district_(thai),district,subdistrict_(thai),subdistrict,province_id,district_id,subdistrict_id
0,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,พระบรมมหาราชวัง,Phra Borom Maha Ratchawang,PROV1,DIST1,SUBDIST1
1,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,วังบูรพาภิรมย์,Wang Burapha Phirom,PROV1,DIST1,SUBDIST2
2,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,วัดราชบพิธ,Wat Ratchabophit,PROV1,DIST1,SUBDIST3
3,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,สำราญราษฎร์,Samran Rat,PROV1,DIST1,SUBDIST4
4,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,ศาลเจ้าพ่อเสือ,San Chao Pho Suea,PROV1,DIST1,SUBDIST5
...,...,...,...,...,...,...,...,...,...,...
7458,10260,กรุงเทพมหานคร,Bangkok,เขตบางนา,Bang Na,บางนาใต้,Bang Na Tai,PROV1,DIST1168,SUBDIST7459
7459,10150,กรุงเทพมหานคร,Bangkok,เขตบางบอน,Bang Bon,บางบอนเหนือ,Bang Bon Nuea,PROV1,DIST1169,SUBDIST7460
7460,10150,กรุงเทพมหานคร,Bangkok,เขตบางบอน,Bang Bon,บางบอนใต้,Bang Bon Tai,PROV1,DIST1169,SUBDIST7461
7461,10150,กรุงเทพมหานคร,Bangkok,เขตบางบอน,Bang Bon,คลองบางพราน,Khlong Bang Phran,PROV1,DIST1169,SUBDIST7462


In [9]:
# coordinates tables
coordinates_df = process_coordinates_tbl('grid_coordinates.json', './data/coordinates.csv')

DataFrame saved to ./data/coordinates.csv


In [10]:
# Stations tables
station_df = process_station_tbl(get_station_data_from_api(), './data/stations.csv')

DataFrame saved to ./data/stations.csv


In [11]:
# Region table
region_df = process_region_tbl('./data/regions.csv')

DataFrame saved to ./data/regions.csv


In [40]:
province_df = pd.read_csv('./data/provinces.csv')

In [55]:
district_df = pd.read_csv('./data/districts.csv')

In [89]:
district_df.rename(columns={'district_id':'id'}, inplace=True)

In [90]:
district_df.to_csv('./data/districts.csv', index=False)

In [ ]:
def location_geocode_sub(geolocator, sub: str, attempts: int = 1, max_attempts: int = 5):
    try:
        location = geolocator.geocode(f"{sub}, Thailand", timeout=10)
        if location:
            print(f"Geocoded {sub}: ({location.latitude}, {location.longitude})")
            return location.latitude, location.longitude
        else:
            print(f"Could not geocode sub-district: {sub}")
            return 0, 0
    except GeocoderTimedOut:
        if attempts < max_attempts:
            print(f"Timeout occurred. Retrying {attempts}/{max_attempts} for sub-district: {sub}")
            return location_geocode(geolocator, sub, attempts + 1, max_attempts)
        else:
            print(f"Geocoding timed out for sub-district: {sub}")
            return 0, 0
    except Exception as e:
        print(f"Error geocoding sub-district {sub}: {e}")
        return 0, 0

In [59]:
for index, row in location_df.iterrows():
    sub = row['subdistrict']
    district = row['district']
    province = row['province']
    latitude, longitude = location_geocode_sub(geolocator, sub, district, province)
    location_df.at[index, 'latitude'] = latitude
    location_df.at[index, 'longitude'] = longitude

Geocoded Phra Borom Maha Ratchawang: (13.7499943, 100.4917502)
Geocoded Wang Burapha Phirom: (13.7456684, 100.5018588)
Geocoded Wat Ratchabophit: (13.7488428, 100.4971789)
Geocoded Samran Rat: (13.7521515, 100.5044746)
Geocoded San Chao Pho Suea: (13.7538614, 100.4982514)
Geocoded Sao Chingcha: (13.7518891, 100.5012654)
Geocoded Bowon Niwet: (13.7598162, 100.5007356)
Geocoded Talat Yot: (13.7603321, 100.4978979)
Geocoded Chana Songkhram: (13.7608948, 100.4951566)
Geocoded Ban Phan Thom: (13.7622495, 100.5016047)
Geocoded Bang Khun Phrom: (13.7670199, 100.5016237)
Geocoded Wat Sam Phraya: (13.7658186, 100.499078)
Geocoded Dusit: (13.7705391, 100.5126071)
Could not geocode sub-district: Wachiraphayaban
Could not geocode sub-district: Suan Chit Lada
Geocoded Si Yaek Maha Nak: (13.7535219, 100.5178588)
Geocoded Thanon Nakhon Chai Si: (13.7819232, 100.517993)
Geocoded Krathum Rai: (13.8117432, 100.931654)
Geocoded Nong Chok: (13.8589972, 100.8640355)
Geocoded Khlong Sip: (13.9038984, 100.81

In [87]:
location_tmp = location_df.copy()
location_tmp.to_csv('./data/subdistricts.csv', index=False)

In [23]:
# Manually add lat long for missing districts
subdistrict_df = process_subdistrict_tbl(location_df)

DataFrame saved to subdistricts.csv


In [77]:
location_df = location_tmp.copy()

In [83]:
district_df

,district_id,postal_code,name_th,name_en,province_id,latitude,longitude,coordinate_id,created_at,updated_at
0,DIST1,10200,เขตพระนคร,Khet Phra Nakhon,PROV1,13.752617,100.493363,COOR1576,2025-10-20 23:53:00,2025-10-20 23:53:00
1,DIST2,10300,เขตดุสิต,Khet Dusit,PROV1,13.770546,100.512608,COOR1641,2025-10-20 23:53:00,2025-10-20 23:53:00
2,DIST3,10530,เขตหนองจอก,Khet Nong Chok,PROV1,13.856310,100.863075,COOR1822,2025-10-20 23:53:00,2025-10-20 23:53:00
3,DIST4,10500,เขตบางรัก,Khet Bang Rak,PROV1,13.723950,100.515960,COOR1640,2025-10-20 23:53:00,2025-10-20 23:53:00
4,DIST5,10220,เขตบางเขน,Khet Bang Khen,PROV1,13.875588,100.596928,COOR1641,2025-10-20 23:53:00,2025-10-20 23:53:00
...,...,...,...,...,...,...,...,...,...,...
1164,DIST1165,10260,เขตพระโขนง,Phra Khanong,PROV1,13.702364,100.601546,COOR1640,2025-10-20 23:53:00,2025-10-20 23:53:00
1165,DIST1166,10250,เขตสวนหลวง,Suan Luang,PROV1,13.719466,100.612949,COOR1699,2025-10-20 23:53:00,2025-10-20 23:53:00
1166,DIST1167,10240,เขตสะพานสูง,Saphan Sung,PROV1,13.768464,100.685422,COOR1699,2025-10-20 23:53:00,2025-10-20 23:53:00
1167,DIST1168,10260,เขตบางนา,Bang Na,PROV1,13.673093,100.606881,COOR1699,2025-10-20 23:53:00,2025-10-20 23:53:00


In [85]:
for idx, row in location_df.iterrows():
    if row['latitude'] == 0 or row['longitude'] == 0:
        lat = district_df.loc[district_df['name_en'] == row['district'], 'latitude'].values
        lon = district_df.loc[district_df['name_en'] == row['district'], 'longitude'].values
        if len(lat) > 0 and len(lon) > 0:
            lat = lat[0]
            lon = lon[0]
        else:
            lat = 0
            lon = 0
        location_df.at[idx, 'latitude'] = lat
        location_df.at[idx, 'longitude'] = lon

In [86]:
location_df

,postal_code,province_(thai),province,district_(thai),district,subdistrict_(thai),subdistrict,province_id,district_id,subdistrict_id,latitude,longitude
0,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,พระบรมมหาราชวัง,Phra Borom Maha Ratchawang,PROV1,DIST1,SUBDIST1,13.749994,100.491750
1,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,วังบูรพาภิรมย์,Wang Burapha Phirom,PROV1,DIST1,SUBDIST2,13.745668,100.501859
2,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,วัดราชบพิธ,Wat Ratchabophit,PROV1,DIST1,SUBDIST3,13.748843,100.497179
3,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,สำราญราษฎร์,Samran Rat,PROV1,DIST1,SUBDIST4,13.752152,100.504475
4,10200,กรุงเทพมหานคร,Bangkok,เขตพระนคร,Khet Phra Nakhon,ศาลเจ้าพ่อเสือ,San Chao Pho Suea,PROV1,DIST1,SUBDIST5,13.753861,100.498251
...,...,...,...,...,...,...,...,...,...,...,...,...
7458,10260,กรุงเทพมหานคร,Bangkok,เขตบางนา,Bang Na,บางนาใต้,Bang Na Tai,PROV1,DIST1168,SUBDIST7459,13.669538,100.605229
7459,10150,กรุงเทพมหานคร,Bangkok,เขตบางบอน,Bang Bon,บางบอนเหนือ,Bang Bon Nuea,PROV1,DIST1169,SUBDIST7460,13.655620,100.384870
7460,10150,กรุงเทพมหานคร,Bangkok,เขตบางบอน,Bang Bon,บางบอนใต้,Bang Bon Tai,PROV1,DIST1169,SUBDIST7461,13.646175,100.388024
7461,10150,กรุงเทพมหานคร,Bangkok,เขตบางบอน,Bang Bon,คลองบางพราน,Khlong Bang Phran,PROV1,DIST1169,SUBDIST7462,13.677826,100.425429


/var/folders/fc/mgc_t1mn5kq1njs8pzw1jrn00000gp/T/ipykernel_26904/116811389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdistrict_df.rename(columns={


/var/folders/fc/mgc_t1mn5kq1njs8pzw1jrn00000gp/T/ipykernel_26904/326680940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdistrict_df['full_address_eng'] = subdistrict_df['name_en'] + ', ' + district_df.set_index('id').loc[subdistrict_df['district_id'], 'name_en'].values + ', ' + province_df.set_index('id').loc[subdistrict_df['province_id'], 'name_en'].values + ', Thailand'
/var/folders/fc/mgc_t1mn5kq1njs8pzw1jrn00000gp/T/ipykernel_26904/326680940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [96]:
subdistrict_df = adding_date_cols(subdistrict_df)
save_df_to_csv(subdistrict_df, './data/subdistricts.csv')

DataFrame saved to ./data/subdistricts.csv


/Users/trung.tran/Library/Mobile Documents/com~apple~CloudDocs/Documents/Personal/Project/airflow/scripts/database/references_tbl/manual/create_tbl.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = current_time


In [98]:
subdistrict_df = find_nearest_coordinates(subdistrict_df, coordinates_df)

/Users/trung.tran/Library/Mobile Documents/com~apple~CloudDocs/Documents/Personal/Project/airflow/scripts/database/references_tbl/manual/func.py:123: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'COOR1576' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  location_df.at[idx, 'coordinate_id'] = coordinates_df.at[nearest_idx, 'id']


In [100]:
save_df_to_csv(subdistrict_df, './data/subdistricts.csv')

DataFrame saved to ./data/subdistricts.csv
